In [ ]:
from sshtunnel import SSHTunnelForwarder
import psycopg2
import pandas as pd
 
server = SSHTunnelForwarder(
    ssh_address_or_host=('10.12.0.8', 2043),
    ssh_username='root', 
    ssh_password = 'ynby20250625', 
 
    
    remote_bind_address=('localhost' , 5432)) 
server.start()
conn = psycopg2.connect(database = 'FAERS20-24', 
                            user = 'postgres',     
                            password = 'woshipostgres', 
                            host = '127.0.0.1',   
                            port = server.local_bind_port)

,a,b,c,d,ror
0,3,21,64388,6916647,15.34591


In [ ]:
def get_drugids():
    sql = ("SELECT DISTINCT drugbank_id FROM drug21_24_map ORDER BY drugbank_id ASC")
    drugids = pd.read_sql(sql, con = conn)
    for i in drugids['drugbank_id']:
        return i

In [35]:
def get_drug_reaction(drugid):
    sql = (f"SELECT primaryid, reactid FROM cdir_fo WHERE drugbank_id = '{drugid}';")
    drug_reaction = pd.read_sql(sql, con = conn)
    #print(drug_reaction)
    primaryid, reactid = drug_reaction.iloc[:, 0], drug_reaction.iloc[:, 1]
    return primaryid, reactid
pid, rid = get_drug_reaction('DB14500')

In [ ]:
def count_drug(drugid):
    '''
    Figure out the value 'a+c'(the sum of recordings of all reactions 
    triggered by the given drug) in the Disproportional Method
    '''
    sql = (f"SELECT count FROM drugcount WHERE drugbank_id = '{drugid}';")
    drugcount = pd.read_sql(sql, con=conn)
    return drugcount['count'].values[0]

count_drug('DB14500')

18004

In [ ]:
def count_react(reactid):
    '''
    Figure out the value 'a+b'(the sum of recordings of the given reaction 
    ) in the Disproportional Method
    '''
    sql = (f"SELECT count FROM reactcount WHERE reactid = '{reactid}';")
    reactcount = pd.read_sql(sql, con=conn)
    return reactcount['count'].values[0]

count_react('254761')

123912

In [ ]:
def count_a(drugid, reactid):
    '''
    Figure out the value 'a' (the count of recordings associating 
    with given drug and focusing adverse reaction) 
    in the Disproportional Method
    '''
    sql = (f"SELECT count(*) FROM cdir_fo WHERE drugbank_id = '{drugid}' AND reactid = '{reactid}';")
    reactcount = pd.read_sql(sql, con=conn)
    return reactcount['count'].values[0]

count_a('DB14500', '254761')

777

In [ ]:

sql = ("WITH target_reports AS (SELECT DISTINCT primaryid, caseid FROM drug21_24_map WHERE drugbank_id = 'DB00054'),reaction_reports AS ( SELECT DISTINCT primaryid, caseid FROM reaction WHERE id = '317002'),all_reports AS ( SELECT DISTINCT primaryid, caseid FROM drug21_24_map UNION SELECT DISTINCT primaryid, caseid FROM reaction),combined AS ( SELECT ar.primaryid, ar.caseid, (tr.primaryid IS NOT NULL) AS has_drug, (rr.primaryid IS NOT NULL) AS has_reaction FROM all_reports ar LEFT JOIN target_reports tr ON ar.primaryid = tr.primaryid AND ar.caseid = tr.caseid LEFT JOIN reaction_reports rr ON ar.primaryid = rr.primaryid AND ar.caseid = rr.caseid),counts AS ( SELECT COUNT(*) FILTER (WHERE has_drug AND has_reaction) AS a, COUNT(*) FILTER (WHERE has_drug AND NOT has_reaction) AS b, COUNT(*) FILTER (WHERE NOT has_drug AND has_reaction) AS c, COUNT(*) FILTER (WHERE NOT has_drug AND NOT has_reaction) AS d FROM combined) SELECT a, b, c, d, CASE  WHEN b = 0 OR c = 0 THEN NULL ELSE (a * d) * 1.0 / (b * c) END AS ror FROM counts")
e_data_intfc = pd.read_sql(sql,con = conn)

e_data_intfc.head()


In [3]:
server.close()

In [15]:
sql = ("SELECT DISTINCT drugbank_id FROM drug21_24_map ORDER BY drugbank_id ASC")
drugids = pd.read_sql(sql, con = conn)